In [ ]:
import numpy as np
import plotly.express as px

import pickle as pic

In [ ]:
from utils import *
from templates import *
from data_loader import EntireTraceIterator

In [ ]:
traces_path = "..\\..\\acquisition\\1000000_maskshuffle_allrounds\\carto_eB4-Rnd-3-WhiteningAndFullFilter-Masking-Shuffling.mat"
key_path = "..\\..\\acquisition\\1000000_maskshuffle_allrounds\\carto_eB4-Rnd-3-WhiteningAndFullFilter-Masking-Shuffling.log"

In [ ]:
NUM_TRACES = 1_000_000
TRACE_SIZE = 200_000
PER_DIVISION = 250_000
data_loader = EntireTraceIterator(traces_path, key_path, nr_populations=1, nr_scenarios=1, trace_size=TRACE_SIZE, traces_per_division=PER_DIVISION, parse_output="keyshares+perms")

## Vizualisation

In [ ]:
folder = "../leakage_points_orig"
all_plot_labels = []
try:
    with open(f"{folder}/f_rws_perms_sost.pic", "rb") as r:
        sig_rws = pic.load(r) / (KEYROUND_WIDTH_B4 * (KEYROUND_WIDTH_B4 - 1) / 2)
    all_plot_labels.append("RWS perm")
except OSError:
    sig_rws = None
sig_masks_rws = []
for keyround_idx in range(KEYROUND_WIDTH_B4):
    for share_idx in range(NR_SHARES):
        try:
            with open(f"{folder}/f_rws_masks_{keyround_idx}_{share_idx}_sost.pic", "rb") as r:
                sig_masks_rws.append(pic.load(r) / (len(KEY_ALPHABET) * (len(KEY_ALPHABET) - 1) / 2))
            all_plot_labels.append(f"RWS mask {keyround_idx} {share_idx}")
        except OSError:
            pass

try:
    with open(f"{folder}/f_round_perms_sost.pic", "rb") as r:
        sig_round_perm = pic.load(r) / ((KEYROUND_WIDTH_B4 // BLOCK_WIDTH_B4) * ((KEYROUND_WIDTH_B4 // BLOCK_WIDTH_B4) - 1) / 2)
    all_plot_labels.append("Round perm")
except OSError:
    sig_round_perm = None
sig_copy_perms = []
sig_masks_perms = []
for round_idx in range(EARLIEST_ROUND, LATEST_ROUND):
    try:
        with open(f"{folder}/f_copy_perms_{round_idx}_sost.pic", "rb") as r:
            sig_copy_perms.append(pic.load(r) / (BLOCK_WIDTH_B4 * (BLOCK_WIDTH_B4 - 1) / 2))
        all_plot_labels.append(f"Copy perm {round_idx}")
    except OSError:
        pass

for round_idx in range(EARLIEST_ROUND, LATEST_ROUND):
    for block_idx in range(BLOCK_WIDTH_B4):
        for share_idx in range(NR_SHARES):
            try:
                with open(f"{folder}/f_round_masks_{round_idx}_{block_idx}_{share_idx}_sost.pic", "rb") as r:
                    sig_masks_perms.append(pic.load(r) / (len(KEY_ALPHABET) * (len(KEY_ALPHABET) - 1) / 2))
                all_plot_labels.append(f"Round mask {round_idx} {block_idx} {share_idx}")
            except OSError:
                pass


In [ ]:
trace_size = 200_000
for it in range(0, trace_size, trace_size // 4):
    everything = ([sig_rws] if sig_rws is not None else []) + sig_masks_rws + ([sig_round_perm] if sig_round_perm is not None else []) + sig_copy_perms + sig_masks_perms
    everything = [sost[it:it+trace_size // 4] for sost in everything]
    fig = px.line(y=everything)
    plot_labs = {f'wide_variable_{i}': l for i, l in enumerate(all_plot_labels)}
    fig.for_each_trace(lambda t: t.update(name = plot_labs[t.name], legendgroup = plot_labs[t.name], 
                                        hovertemplate = t.hovertemplate.replace(t.name, plot_labs[t.name])))
    fig.show()
    break

# Focus on smaller windows of the trace

In [ ]:
RWS_WINDOW = np.arange(1100, 21500)
ROUND_PERM_WINDOW = np.concatenate((
        np.arange( 21750,  21800),
        np.arange( 33925,  33975),
        np.arange( 46250,  46300),
        np.arange( 58550,  25600),
        np.arange( 70860,  70910),
        np.arange( 83180,  83230),
        np.arange( 95480,  95530),
        np.arange(107800, 107850),
        np.arange(120110, 120160),
        np.arange(132415, 132465),
        np.arange(144725, 144775),
        np.arange(157040, 157090),
        np.arange(169350, 169400),
        np.arange(181660, 181710),
        np.arange(193960, 194010),
    ), axis=0)
ROUND_WINDOWS = np.stack((
    np.arange( 21950,  23350), # Round 0
    np.arange( 34250,  35650), # Round 1
    np.arange( 46550,  47950), # Round 2
    np.arange( 58850,  60250), # Round 3
    np.arange( 71150,  72550), # Round 4
    np.arange( 83500,  84900), # Round 5
    np.arange( 95800,  97200), # Round 6
    np.arange(108100, 109500), # Round 7
    np.arange(120400, 121800), # Round 8
    np.arange(132700, 134100), # Round 9
    np.arange(145025, 146425), # Round 10
    np.arange(157350, 158750), # Round 11
    np.arange(169650, 171050), # Round 12
    np.arange(181950, 183350), # Round 13
), axis=0)
traces_rws_only = np.zeros((NUM_TRACES, len(RWS_WINDOW)), dtype=np.float16)
traces_round_perm_only = np.zeros((NUM_TRACES, len(ROUND_PERM_WINDOW)), dtype=np.float16)
traces_rounds_i_only = np.zeros((ROUND_WINDOWS.shape[0], NUM_TRACES, ROUND_WINDOWS.shape[1]), dtype=np.float16)
for j, (_, traces, _, _) in enumerate(data_loader((0,), (0,))):
    traces = traces[0][0]
    traces_rws_only[j * PER_DIVISION:j * PER_DIVISION + traces.shape[0]] = traces[:, RWS_WINDOW]
    traces_round_perm_only[j * PER_DIVISION:j * PER_DIVISION + traces.shape[0]] = traces[:, ROUND_PERM_WINDOW]
    for i in range(len(ROUND_WINDOWS)):
        traces_rounds_i_only[i, j * PER_DIVISION:j * PER_DIVISION + traces.shape[0]] = traces[:, ROUND_WINDOWS[i]]
    del traces

In [ ]:
np.where(np.all(traces_rws_only == 0, axis=-1)), np.where(np.all(traces_round_perm_only == 0, axis=-1)), np.where(np.all(traces_rounds_i_only == 0, axis=-1))

(array([399999], dtype=int64),)

In [ ]:
traces_rws_only = np.concatenate((traces_rws_only[:399999], traces_rws_only[400000:]), axis=0)
with open("../traces_and_splits/traces_rws_only.pic", "wb") as w:
    pic.dump(traces_rws_only, w)

In [ ]:
traces_round_perm_only = np.concatenate((traces_round_perm_only[:399999], traces_round_perm_only[400000:]), axis=0)
with open("../traces_and_splits/traces_round_perm_only.pic", "wb") as w:
    pic.dump(traces_round_perm_only, w)

In [ ]:
for i in range(len(ROUND_WINDOWS)):
    wout_empty_traces = np.concatenate((traces_rounds_i_only[i, :399999], traces_rounds_i_only[i, 400000:]), axis=0)
    with open(f"../traces_and_splits/traces_round_{i}_only.pic", "wb") as w:
        pic.dump(wout_empty_traces, w)